In [ ]:
import torch
import torchvision
import pandas as pd
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!curl -L "https://app.roboflow.com/ds/Auk0FyQnUp?key=nc7UIyERZJ" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   892  100   892    0     0   3417      0 --:--:-- --:--:-- --:--:--  3417
100 62.4M  100 62.4M    0     0  46.2M      0  0:00:01  0:00:01 --:--:-- 84.1M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/IMG_2289_jpeg_jpg.rf.16d9ccd5bcb468eb128107d339e765eb.jpg  
 extracting: test/IMG_2301_jpeg_jpg.rf.07ab307ecf5addb951a1b0ea676f944a.jpg  
 extracting: test/IMG_2319_jpeg_jpg.rf.40a9d4d5b8c1ebadb2dde5de0938f806.jpg  
 extracting: test/IMG_2347_jpeg_jpg.rf.caad6d0e2a3842a796664df8dff12802.jpg  
 extracting: test/IMG_2354_jpeg_jpg.rf.e608f644d491a80acf76bb0062e824be.jpg  
 extracting: test/IMG_2371_jpeg_jpg.rf.8fe3621c32b29a52f7bff044c483ce3e.jpg  
 extracting: test/IMG_2379_jpeg_jpg.rf.74549ec540a9fd45cb3023500f190e68.jpg  
 extracting: test/IM

In [ ]:
import torch
import torchvision
from torchvision import datasets, models
from torchvision.transforms import functional as FT
from torchvision.transforms import transforms as T
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split, Dataset
import copy
import math
import cv2
from PIL import Image
import albumentations as A
import cv2

import matplotlib.pyplot as plt
# %matplotlib inline

In [ ]:
!pip install pycocotools
from pycocotools.coco import COCO

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from albumentations.pytorch import ToTensorV2
def get_transforms(train=False):
    if train:
        transform = A.Compose([
            A.Resize(600, 600), 
            A.HorizontalFlip(p=0.3),
            A.VerticalFlip(p=0.3),
            A.RandomBrightnessContrast(p=0.1),
            A.ColorJitter(p=0.1),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='coco'))
    else:
        transform = A.Compose([
            A.Resize(600, 600), 
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='coco'))
    return transform

In [ ]:
class AquariumDetection(datasets.VisionDataset):
    def __init__(self, root, split='train', transform=None, target_transform=None, transforms=None):
        
        super().__init__(root, transforms, transform, target_transform)
        self.split = split
        self.coco = COCO(os.path.join(root, split, "_annotations.coco.json")) # annotatiosns
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.ids = [id for id in self.ids if (len(self._load_target(id)) > 0)]
    
    def _load_image(self, id: int):
        path = self.coco.loadImgs(id)[0]['file_name']
        image = cv2.imread(os.path.join(self.root, self.split, path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image
    def _load_target(self, id):
        return self.coco.loadAnns(self.coco.getAnnIds(id))
    
    def __getitem__(self, index):
        id = self.ids[index]
        image = self._load_image(id)
        target = self._load_target(id)
        target = copy.deepcopy(self._load_target(id))
        
        boxes = [t['bbox'] + [t['category_id']] for t in target] 
        if self.transforms is not None:
            transformed = self.transforms(image=image, bboxes=boxes)
        
        image = transformed['image']
        boxes = transformed['bboxes']
        
        new_boxes = [] #  xywh to xyxy
        for box in boxes:
            xmin = box[0]
            xmax = xmin + box[2]
            ymin = box[1]
            ymax = ymin + box[3]
            new_boxes.append([xmin, ymin, xmax, ymax])
        
        boxes = torch.tensor(new_boxes, dtype=torch.float32)
        
        targ = {} 
        targ['boxes'] = boxes
        targ['labels'] = torch.tensor([t['category_id'] for t in target], dtype=torch.int64)
        targ['image_id'] = torch.tensor([t['image_id'] for t in target])
        targ['area'] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        targ['iscrowd'] = torch.tensor([t['iscrowd'] for t in target], dtype=torch.int64)
        return image.div(255), targ #scalling
        
    def __len__(self):
        return len(self.ids)

In [ ]:
path =""
coc = COCO(os.path.join(path, "train", "_annotations.coco.json"))
categories = coc.cats
n_classes = len(categories.keys())
n_classes, categories

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


(8,
 {0: {'id': 0, 'name': 'Fishes', 'supercategory': 'none'},
  1: {'id': 1, 'name': 'fish', 'supercategory': 'Fishes'},
  2: {'id': 2, 'name': 'jellyfish', 'supercategory': 'Fishes'},
  3: {'id': 3, 'name': 'penguin', 'supercategory': 'Fishes'},
  4: {'id': 4, 'name': 'puffin', 'supercategory': 'Fishes'},
  5: {'id': 5, 'name': 'shark', 'supercategory': 'Fishes'},
  6: {'id': 6, 'name': 'starfish', 'supercategory': 'Fishes'},
  7: {'id': 7, 'name': 'stingray', 'supercategory': 'Fishes'}})

In [ ]:
classes = []
for i in categories.items():
    classes.append(i[1]["name"])

In [ ]:
train_dataset = AquariumDetection(root=path, transforms=get_transforms(True))
val_dataset = AquariumDetection(root=path, split="valid", transforms=get_transforms(False))
test_dataset = AquariumDetection(root=path, split="test", transforms=get_transforms(False))

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
len(train_dataset), len(val_dataset), len(test_dataset)

(894, 127, 63)

In [ ]:
model = models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features # changing head
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, n_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth
100%|██████████| 74.2M/74.2M [00:01<00:00, 69.3MB/s]


In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=collate_fn
)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=collate_fn
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
images,targets = next(iter(train_loader))
images = list(image for image in images)
targets = [{k:v for k, v in t.items()} for t in targets]
output = model(images, targets)
output

{'loss_classifier': tensor(2.0697, grad_fn=<NllLossBackward0>),
 'loss_box_reg': tensor(0.7019, grad_fn=<DivBackward0>),
 'loss_objectness': tensor(0.3135, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'loss_rpn_box_reg': tensor(0.0377, grad_fn=<DivBackward0>)}

In [ ]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, nesterov=True, weight_decay=1e-4)

In [ ]:
from tqdm import tqdm
import sys

In [ ]:

num_epochs = 10
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

for epoch in range(num_epochs):
    model.to(device)
    model.train()
    header = 'Epoch: [{}]'.format(epoch)

    all_losses = []
    all_losses_dict = []
    
    for images, targets in tqdm(train_loader):
        images = list(image.to(device) for image in images)
        targets = [{k: torch.tensor(v).to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        loss_dict_append = {k: v.item() for k, v in loss_dict.items()}
        losses_reduced = sum(loss for loss in loss_dict.values())

        loss_value = losses_reduced.item()
        
        all_losses.append(loss_value)
        all_losses_dict.append(loss_dict_append)
        
        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    
    all_losses_dict = pd.DataFrame(all_losses_dict)
    print("Epoch {}: lr: {:.6f} loss: {:.6f}".format(epoch, optimizer.param_groups[0]["lr"], np.mean(all_losses) ))


100%|██████████| 224/224 [23:40<00:00,  6.34s/it]


Epoch 0: lr: 0.010000 loss: 0.885640


100%|██████████| 224/224 [22:54<00:00,  6.14s/it]


Epoch 1: lr: 0.010000 loss: 0.732145


100%|██████████| 224/224 [21:58<00:00,  5.89s/it]


...

Epoch 7: lr: 0.010000 loss: 0.604087


100%|██████████| 224/224 [19:32<00:00,  5.24s/it]


Epoch 8: lr: 0.010000 loss: 0.597468


100%|██████████| 224/224 [19:23<00:00,  5.19s/it]

Epoch 9: lr: 0.010000 loss: 0.584433


In [ ]:
import plotly.graph_objects as go
y=[]
x = [i for i in range(num_epochs)]
y = all_losses
fig = go.Figure()
fig.add_trace(go.Scatter(x=x,y=y,mode='lines', name='lines'))
fig.update_layout(title='Loss vs Epochs',xaxis_title='Epochs',yaxis_title='Loss')
fig.show()